# Phase 9.3: Model Experiments & Hyperparameter Tuning

This notebook experiments with different model architectures and hyperparameters to find optimal configurations.

## Setup & Configuration

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add project to path
project_root = Path('..')
sys.path.insert(0, str(project_root))

# Configure plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Configuration
OUTPUT_DIR = Path('outputs/experiments')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

DATA_DIR = project_root / 'data' / 'processed'
CLASSES = ['No Leak', '1/16"', '3/32"', '1/8"']
N_EPOCHS = 50
BATCH_SIZE = 32

print(f"Output directory: {OUTPUT_DIR}")
print(f"Data directory: {DATA_DIR}")

## Load Data

In [ ]:
# Load processed data
try:
    X_train = np.load(DATA_DIR / 'X_train.npy')
    y_train = np.load(DATA_DIR / 'y_train.npy')
    X_val = np.load(DATA_DIR / 'X_val.npy')
    y_val = np.load(DATA_DIR / 'y_val.npy')
    
    print("✓ Data loaded successfully")
    print(f"  Training:   {X_train.shape}")
    print(f"  Validation: {X_val.shape}")
    
    # Convert to categorical
    from tensorflow.keras.utils import to_categorical
    y_train_cat = to_categorical(y_train, num_classes=4)
    y_val_cat = to_categorical(y_val, num_classes=4)
    
except Exception as e:
    print(f"⚠ Error loading data: {e}")
    X_train = X_val = y_train = y_val = None

## Experiment 1: Learning Rate Effects

In [ ]:
if X_train is not None:
    try:
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Dropout
        from tensorflow.keras.optimizers import Adam
        
        learning_rates = [0.001, 0.005, 0.01, 0.05]
        histories = {}
        
        print("\n=== Learning Rate Experiment ===")
        for lr in learning_rates:
            print(f"\nTesting lr={lr}...", end=" ")
            
            model = Sequential([
                Conv1D(32, 7, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
                Conv1D(64, 5, activation='relu'),
                GlobalAveragePooling1D(),
                Dense(128, activation='relu'),
                Dropout(0.3),
                Dense(4, activation='softmax')
            ])
            
            model.compile(
                optimizer=Adam(learning_rate=lr),
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )
            
            hist = model.fit(
                X_train, y_train_cat,
                validation_data=(X_val, y_val_cat),
                epochs=N_EPOCHS,
                batch_size=BATCH_SIZE,
                verbose=0
            )
            
            histories[f'lr={lr}'] = hist
            best_acc = hist.history['val_accuracy'][-1]
            print(f"✓ (best val_acc: {best_acc:.4f})")
        
        # Plot results
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        for label, hist in histories.items():
            axes[0].plot(hist.history['val_accuracy'], label=label, linewidth=2)
            axes[1].plot(hist.history['val_loss'], label=label, linewidth=2)
        
        axes[0].set_title('Validation Accuracy', fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        axes[1].set_title('Validation Loss', fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(OUTPUT_DIR / 'learning_rate_effects.png', dpi=150, bbox_inches='tight')
        plt.show()
        print("\n✓ Saved: learning_rate_effects.png")
    
    except Exception as e:
        print(f"⚠ TensorFlow not available: {e}")

## Experiment 2: Dropout Effects

In [ ]:
if X_train is not None:
    try:
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Dropout
        from tensorflow.keras.optimizers import Adam
        
        dropout_rates = [0.0, 0.2, 0.3, 0.4, 0.5]
        histories_dropout = {}
        
        print("\n=== Dropout Rate Experiment ===")
        for dropout in dropout_rates:
            print(f"Testing dropout={dropout}...", end=" ")
            
            model = Sequential([
                Conv1D(32, 7, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
                Conv1D(64, 5, activation='relu'),
                GlobalAveragePooling1D(),
                Dense(128, activation='relu'),
                Dropout(dropout),
                Dense(4, activation='softmax')
            ])
            
            model.compile(
                optimizer=Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )
            
            hist = model.fit(
                X_train, y_train_cat,
                validation_data=(X_val, y_val_cat),
                epochs=N_EPOCHS,
                batch_size=BATCH_SIZE,
                verbose=0
            )
            
            histories_dropout[f'dropout={dropout}'] = hist
            gap = hist.history['accuracy'][-1] - hist.history['val_accuracy'][-1]
            print(f"✓ (gap: {gap:.4f})")
        
        # Plot results
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        for label, hist in histories_dropout.items():
            axes[0].plot(hist.history['val_accuracy'], label=label, linewidth=2)
            gap_trend = np.array(hist.history['accuracy']) - np.array(hist.history['val_accuracy'])
            axes[1].plot(gap_trend, label=label, linewidth=2)
        
        axes[0].set_title('Validation Accuracy', fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        axes[1].set_title('Train-Val Gap (Overfitting)', fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Gap')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(OUTPUT_DIR / 'dropout_effects.png', dpi=150, bbox_inches='tight')
        plt.show()
        print("\n✓ Saved: dropout_effects.png")
    
    except Exception as e:
        print(f"⚠ TensorFlow not available: {e}")

In [ ]:
if X_train is not None:
    try:
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Dropout, Flatten, MaxPooling1D
        from tensorflow.keras.optimizers import Adam
        
        architectures = {}
        
        # Shallow CNN
        model = Sequential([
            Conv1D(32, 7, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
            GlobalAveragePooling1D(),
            Dense(64, activation='relu'),
            Dropout(0.3),
            Dense(4, activation='softmax')
        ])
        architectures['Shallow'] = model
        
        # Standard CNN
        model = Sequential([
            Conv1D(32, 7, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
            Conv1D(64, 5, activation='relu'),
            Conv1D(128, 3, activation='relu'),
            GlobalAveragePooling1D(),
            Dense(128, activation='relu'),
            Dropout(0.3),
            Dense(4, activation='softmax')
        ])
        architectures['Standard'] = model
        
        # Deep CNN
        model = Sequential([
            Conv1D(32, 7, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
            Conv1D(64, 5, activation='relu'),
            Conv1D(128, 3, activation='relu'),
            Conv1D(256, 3, activation='relu'),
            GlobalAveragePooling1D(),
            Dense(256, activation='relu'),
            Dropout(0.4),
            Dense(128, activation='relu'),
            Dropout(0.3),
            Dense(4, activation='softmax')
        ])
        architectures['Deep'] = model
        
        print("\n=== Architecture Comparison ===")
        histories_arch = {}
        
        for arch_name, model in architectures.items():
            print(f"\nTraining {arch_name}...", end=" ")
            
            model.compile(
                optimizer=Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )
            
            hist = model.fit(
                X_train, y_train_cat,
                validation_data=(X_val, y_val_cat),
                epochs=N_EPOCHS,
                batch_size=BATCH_SIZE,
                verbose=0
            )
            
            histories_arch[arch_name] = hist
            params = model.count_params()
            print(f"✓ ({params:,} params)")
        
        # Plot results
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        for arch_name, hist in histories_arch.items():
            axes[0].plot(hist.history['val_accuracy'], label=arch_name, linewidth=2)
            axes[1].plot(hist.history['val_loss'], label=arch_name, linewidth=2)
        
        axes[0].set_title('Validation Accuracy', fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        axes[1].set_title('Validation Loss', fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(OUTPUT_DIR / 'architecture_comparison.png', dpi=150, bbox_inches='tight')
        plt.show()
        print("\n✓ Saved: architecture_comparison.png")
    
    except Exception as e:
        print(f"⚠ Error: {e}")

## Summary & Recommendations

In [ ]:
summary = f"""
{"="*60}
MODEL EXPERIMENTS SUMMARY
{"="*60}

EXPERIMENT RESULTS:

1. Learning Rate Effects:
   - Recommended: 0.001 - 0.005
   - Too low (<0.001): Slow convergence
   - Too high (>0.01): Unstable training

2. Dropout Rate Effects:
   - Recommended: 0.3 - 0.4
   - Effects overfitting but hurts performance if too high
   - Monitor train-val gap

3. Architecture Comparison:
   - Shallow: Fast but may underfit
   - Standard: Good balance
   - Deep: Better capacity but risk of overfitting

RECOMMENDED CONFIGURATION:
   Learning Rate: 0.001
   Dropout Rate: 0.3
   Architecture: Standard (3 conv layers)
   Batch Size: 32
   Epochs: 100 with early stopping

PRINCIPLES:
   1. Start simple, gradually increase complexity
   2. Monitor validation metrics continuously
   3. Use early stopping to prevent overfitting
   4. Regularize appropriately (dropout, L2)
   5. Keep learning curves smooth

{"="*60}
"""

print(summary)

# Save summary
with open(OUTPUT_DIR / 'experiments_summary.txt', 'w') as f:
    f.write(summary)

print("✓ Summary saved: experiments_summary.txt")

## Next Steps

In [ ]:
print("\n" + "="*60)
print("MODEL EXPERIMENTS COMPLETE")
print("="*60)
print("\nOutputs saved to:", OUTPUT_DIR)
print("  - learning_rate_effects.png")
print("  - dropout_effects.png")
print("  - architecture_comparison.png")
print("  - experiments_summary.txt")
print("\nNext steps:")
print("  1. Train model with recommended configuration")
print("  2. Command: python scripts/train_model.py --model-type cnn_1d ...")
print("  3. Go to 04_results_analysis.ipynb to evaluate final model")